### Generically large neural networks can express any labeling of the training data

原文舉例說可以用包含p = 2n + d個參數的2層ReLU網絡來模擬n個d維樣本的任何標籤可能，但這只是一個網絡表達能力的理論上界，作者沒有繼續討論要達到這個上界需要的條件，以及p、n、d的大小關係對論文核心問題也就是網絡generalization performance的影響。

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
num_class = 2
batch_size = 100
learning_rate = 0.0001
epochs = 200
num_data = 10000 #n
instance_dimension = 500 #d
num_parameters = 2 * num_data + instance_dimension #p

In [3]:
def generate_data():
    return np.random.random_sample(size=[num_data, instance_dimension]), np.random.randint(2, size=num_data)

def relu_network(x, weights, biases):
    layer1 = tf.nn.relu(tf.add(tf.matmul(x, weights['W1']), biases['b1']))
    layer2 = tf.nn.relu(tf.add(tf.matmul(layer1, weights['W2']), biases['b2']))
    output_layer = tf.nn.relu(tf.add(tf.matmul(layer2, weights['out']), biases['out']))
    return output_layer

def onehot(label):
    onehot_label = []
    for l in label:
        lt = [0]*num_class
        lt[l[0]-1] = 1
        onehot_label.append(lt)
    return np.array(onehot_label)

weights = {
    'W1': tf.Variable(tf.random_normal([instance_dimension, num_parameters], stddev=0.01)),
    'W2': tf.Variable(tf.random_normal([num_parameters, num_parameters], stddev=0.01)),
    'out': tf.Variable(tf.random_normal([num_parameters, num_class], stddev=0.01))
}
biases = {
    'b1': tf.Variable(tf.random_normal([num_parameters])),
    'b2': tf.Variable(tf.random_normal([num_parameters])),
    'out': tf.Variable(tf.random_normal([num_class]))
}

In [4]:
x_train, y_train = generate_data()
y_train = np.reshape(y_train, [-1, 1])

In [9]:
def train(x_train, y_train):
    x = tf.placeholder(tf.float32, [None, instance_dimension], name='input')
    y = tf.placeholder(tf.float32, [None, num_class], name='output')
    pred = relu_network(x, weights, biases)
    #loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    y_train = onehot(y_train)

    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())
        myIdx =  np.random.permutation(num_data)
        for epoch in xrange(epochs):
            num_batch = num_data / batch_size
            for i in xrange(num_batch):
                x_batch = x_train[myIdx[i*batch_size:(i+1)*batch_size],:]
                y_batch = x_train[myIdx[i*batch_size:(i+1)*batch_size],:]
                sess.run(optimizer, feed_dict={x: x_train, y: y_train})
            if (epoch+1) % 10 == 0:
                loss_temp = sess.run(loss, feed_dict={x: x_train, y: y_train}) 
                accuracy_temp = accuracy.eval({x: x_train, y: y_train})
                print "(epoch {})".format(epoch+1) 
                print "[Loss / Tranining Accuracy] {:05.4f} / {:05.4f}".format(loss_temp, accuracy_temp)
                print " "

In [10]:
train(x_train, y_train) #Normal

ResourceExhaustedError: OOM when allocating tensor with shape[10000,20500]
	 [[Node: Add_7 = Add[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](MatMul_7, Variable_4/read)]]

Caused by op u'Add_7', defined at:
  File "/usr/lib64/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/lib64/python2.7/site-packages/zmq/eventloop/ioloop.py", line 151, in start
    super(ZMQIOLoop, self).start()
  File "/usr/lib64/python2.7/site-packages/tornado/ioloop.py", line 888, in start
    handl